Import required libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Pull the content from the Wiki page using bs4

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

Convert content to data list

In [3]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

Create dataframe from list data

In [4]:
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Drop rows with non content

In [6]:
df = df.dropna()

Drop "Not Assigned" rows

In [7]:
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [8]:
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Group by Postcode and convert groupby values as comma-separated strings into new dataframe called df2

In [9]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['PostalCode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [10]:
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Perform some validation on data

In [11]:
df2[df2.Borough == 'Downtown Toronto']

,PostalCode,Borough,Neighborhood
50,M4W,Downtown Toronto,Rosedale
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town"
52,M4Y,Downtown Toronto,Church and Wellesley
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"
54,M5B,Downtown Toronto,"Garden District, Ryerson"
55,M5C,Downtown Toronto,St. James Town
56,M5E,Downtown Toronto,Berczy Park
57,M5G,Downtown Toronto,Central Bay Street
58,M5H,Downtown Toronto,"Adelaide, King, Richmond"
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station"


Save df2 for further use later as comma-separated value file

In [12]:
df2.to_csv('task1.csv', index=False)

Print new dataframe (df2) shape

In [13]:
df2.shape

(102, 3)